In [1]:
# import packages
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [141]:
# Upload the dataset csv file to the session storage 
# using the file explorer on the left
# right-click to get the file path
# then load it to the notebook
wbd = pd.read_csv('/content/wb-disc1.csv')

The variables related to discrimination laws that we are interested in are: 

* SG.LEG.SXHR.EM : There is legislation on sexual harassment in 
employment (1=yes; 0=no)
* SG.DML.PRGW : Dismissal of pregnant workers is prohibited (1=yes; 0=no)
* SG.LAW.EQRM.WK : Law mandates equal remuneration for females and males for work of equal value (1=yes; 0=no)
* SG.LAW.NODC.HR : The law prohibits discrimination in employment based on gender (1=yes; 0=no)
* SG.LAW.CRDD.GR : The law prohibits discrimination in access to credit based on gender (1=yes; 0=no)
* SG.LAW.NMCN : The law provides for the valuation of nonmonetary contributions (1=yes; 0=no)
* SG.GET.JOBS.EQ : A woman can get a job in the same way as a man (1=yes; 0=no)
* SG.BUS.REGT.EQ : A woman can register a business in the same way as a man (1=yes; 0=no)

In [142]:
# quick check
print(wbd.columns, len(wbd.columns))

Index(['CountryName', 'CountryCode', 'Year', 'CountryYear', 'SG.LAW.EQRM.WK',
       'SG.LAW.NODC.HR', 'SG.LAW.CRDD.GR', 'SG.LAW.NMCN', 'SG.DML.PRGW',
       'SG.LEG.SXHR.EM', 'SG.GET.JOBS.EQ', 'SG.BUS.REGT.EQ'],
      dtype='object') 12


We are interested in when the status for each law of interest changed, so we need to generate additional columns to calculate any changes from year to year in each of the features listed above:
* EQRM-CHG
* NODC-CHG
* CRDD-CHG
* NMCN-CHG
* PRGW-CHG
* SXHR-CHG
* JOBS-CHG
* REGT-CHG

Then we can filter the data and look for non-zero values in the -chg columns.

In [143]:
#creating a new column to take the difference in each row for each existing column
wbd['change_EQRM'] = wbd['SG.LAW.EQRM.WK'].diff()
wbd['change_NODC'] = wbd['SG.LAW.NODC.HR'].diff()
wbd['change_SXHR'] = wbd['SG.LEG.SXHR.EM'].diff()
wbd['change_NMCN'] = wbd['SG.LAW.NMCN'].diff()
wbd['change_PRGW'] = wbd['SG.DML.PRGW'].diff()
wbd['change_REGT'] = wbd['SG.BUS.REGT.EQ'].diff()
wbd['change_CRDD'] = wbd['SG.LAW.CRDD.GR'].diff()
wbd['change_JOBS'] = wbd['SG.GET.JOBS.EQ'].diff()

In [144]:
# quick check
print(wbd.head())

   CountryName CountryCode  Year CountryYear  SG.LAW.EQRM.WK  SG.LAW.NODC.HR  \
0  Afghanistan         AFG  1970     AFG1970               0               0   
1  Afghanistan         AFG  1971     AFG1971               0               0   
2  Afghanistan         AFG  1972     AFG1972               0               0   
3  Afghanistan         AFG  1973     AFG1973               0               0   
4  Afghanistan         AFG  1974     AFG1974               0               0   

   SG.LAW.CRDD.GR  SG.LAW.NMCN  SG.DML.PRGW  SG.LEG.SXHR.EM  SG.GET.JOBS.EQ  \
0               1            0            0               0               1   
1               1            0            0               0               1   
2               1            0            0               0               1   
3               1            0            0               0               1   
4               1            0            0               0               1   

   SG.BUS.REGT.EQ  change_EQRM  change_NODC 

In [145]:
# this resulted in NaN values for 1970 because there is no prior record
# let's remove those
wbd = wbd[wbd.Year != 1970]
print(wbd.head())

   CountryName CountryCode  Year CountryYear  SG.LAW.EQRM.WK  SG.LAW.NODC.HR  \
1  Afghanistan         AFG  1971     AFG1971               0               0   
2  Afghanistan         AFG  1972     AFG1972               0               0   
3  Afghanistan         AFG  1973     AFG1973               0               0   
4  Afghanistan         AFG  1974     AFG1974               0               0   
5  Afghanistan         AFG  1975     AFG1975               0               0   

   SG.LAW.CRDD.GR  SG.LAW.NMCN  SG.DML.PRGW  SG.LEG.SXHR.EM  SG.GET.JOBS.EQ  \
1               1            0            0               0               1   
2               1            0            0               0               1   
3               1            0            0               0               1   
4               1            0            0               0               1   
5               1            0            0               0               1   

   SG.BUS.REGT.EQ  change_EQRM  change_NODC 

In [149]:
# create a dataframe without the original binary features
# and remove the additional index columns for later purposes
wb3 = wbd.drop(columns=['change_EQRM', 'change_NMCN', 'change_SXHR', 'change_NODC', 'change_CRDD', 'change_JOBS', 'change_REGT', 'change_PRGW', 'SG.LAW.EQRM.WK', 'SG.LAW.NODC.HR', 'SG.LAW.CRDD.GR', 'SG.LAW.NMCN', 'SG.DML.PRGW', 'SG.LEG.SXHR.EM', 'SG.GET.JOBS.EQ', 'SG.BUS.REGT.EQ'])
wb2 = wbd.drop(columns=['CountryCode','CountryName', 'Year', 'SG.LAW.EQRM.WK', 'SG.LAW.NODC.HR', 'SG.LAW.CRDD.GR', 'SG.LAW.NMCN', 'SG.DML.PRGW', 'SG.LEG.SXHR.EM', 'SG.GET.JOBS.EQ', 'SG.BUS.REGT.EQ'])

In [150]:
# quick check
print(wb2.columns, len(wb2.columns))

Index(['CountryYear', 'change_EQRM', 'change_NODC', 'change_SXHR',
       'change_NMCN', 'change_PRGW', 'change_REGT', 'change_CRDD',
       'change_JOBS'],
      dtype='object') 9


In [151]:
# quick check
print(wb3.columns, len(wb3.columns))

Index(['CountryName', 'CountryCode', 'Year', 'CountryYear'], dtype='object') 4


In [110]:
# quick check
print(wb2.head())

  CountryYear  change_EQRM  change_NODC  change_SXHR  change_NMCN  \
1     AFG1971          0.0          0.0          0.0          0.0   
2     AFG1972          0.0          0.0          0.0          0.0   
3     AFG1973          0.0          0.0          0.0          0.0   
4     AFG1974          0.0          0.0          0.0          0.0   
5     AFG1975          0.0          0.0          0.0          0.0   

   change_PRGW  change_REGT  change_CRDD  change_JOBS  
1          0.0          0.0          0.0          0.0  
2          0.0          0.0          0.0          0.0  
3          0.0          0.0          0.0          0.0  
4          0.0          0.0          0.0          0.0  
5          0.0          0.0          0.0          0.0  


In [184]:
# create an empty dataframe to populate with only the years where laws changed
change = pd.DataFrame()

In [185]:
# now we can design a loop to scan each column and append the empty dataframe 
#with the country-year that experiences a change
for index in range(len(wb2)):
  if wb2.iloc[index,1:].astype(bool).any():
    change = change.append(wb2.iloc[index,:])

In [186]:
# now we have a dataframe with only the years where changes occurred. 
# a 1 indicates that the law was implemented, and a -1 indicates a law was repealed
# 0 means no change in that law
print(change.head())
print(len(change))

   CountryYear  change_EQRM  change_NODC  change_SXHR  change_NMCN  \
18     AFG1988          0.0          0.0          0.0          0.0   
29     AFG1999          0.0          0.0          0.0          0.0   
46     AFG2016          0.0          0.0          1.0          0.0   
78     ALB1996          0.0          1.0          1.0          0.0   
98     ALB2016          1.0          0.0          0.0          0.0   

    change_PRGW  change_REGT  change_CRDD  change_JOBS  
18          1.0          0.0          0.0          0.0  
29         -1.0          0.0          0.0          0.0  
46          0.0          0.0          0.0          0.0  
78          1.0          0.0          0.0          0.0  
98          0.0          0.0          0.0          0.0  
472


In [187]:
# now we can replace the integer values with the categorical values

#making a duplicate just in case
changes = change

# we will get rid of the 0.0 values
changes = changes.replace(to_replace = 0.0, value = "")

# a -1.0 means something was repealed that year
# and a 1.0 means something was implemented that year

changes['change_CRDD'] = changes['change_CRDD'].replace(to_replace = -1.0, value = "Repealed CRDD")
changes['change_CRDD'] = changes['change_CRDD'].replace(to_replace = 1.0, value = "Implemented CRDD")

changes['change_EQRM'] = changes['change_EQRM'].replace(to_replace = -1.0, value = "Repealed EQRM")
changes['change_EQRM'] = changes['change_EQRM'].replace(to_replace = 1.0, value = "Implemented EQRM")

changes['change_NODC'] = changes['change_NODC'].replace(to_replace = -1.0, value = "Repealed NODC")
changes['change_NODC'] = changes['change_NODC'].replace(to_replace = 1.0, value = "Implemented NODC")

changes['change_PRGW'] = changes['change_PRGW'].replace(to_replace = -1.0, value = "Repealed PRGW")
changes['change_PRGW'] = changes['change_PRGW'].replace(to_replace = 1.0, value = "Implemented PRGW")

changes['change_SXHR'] = changes['change_SXHR'].replace(to_replace = -1.0, value = "Repealed SXHR")
changes['change_SXHR'] = changes['change_SXHR'].replace(to_replace = 1.0, value = "Implemented SXHR")

changes['change_NMCN'] = changes['change_NMCN'].replace(to_replace = -1.0, value = "Repealed NMCN")
changes['change_NMCN'] = changes['change_NMCN'].replace(to_replace = 1.0, value = "Implemented NMCN")

changes['change_REGT'] = changes['change_REGT'].replace(to_replace = -1.0, value = "Repealed REGT")
changes['change_REGT'] = changes['change_REGT'].replace(to_replace = 1.0, value = "Implemented REGT")

changes['change_JOBS'] = changes['change_JOBS'].replace(to_replace = -1.0, value = "Repealed JOBS")
changes['change_JOBS'] = changes['change_JOBS'].replace(to_replace = 1.0, value = "Implemented JOBS")

In [188]:
# quick check
print(changes.head())
print(len(changes))

   CountryYear       change_EQRM       change_NODC       change_SXHR  \
18     AFG1988                                                         
29     AFG1999                                                         
46     AFG2016                                      Implemented SXHR   
78     ALB1996                    Implemented NODC  Implemented SXHR   
98     ALB2016  Implemented EQRM                                       

   change_NMCN       change_PRGW change_REGT change_CRDD change_JOBS  
18              Implemented PRGW                                      
29                 Repealed PRGW                                      
46                                                                    
78              Implemented PRGW                                      
98                                                                    
472


In [126]:
# let's melt the data to consolidate the columns
# while retaining the country-year as the index
# and renaming the values to the change that occurred
change_melt = changes.melt(id_vars=['CountryYear'], value_name="Change")
print(change_melt.head())

  CountryYear     variable            Change
0     AFG1988  change_EQRM                  
1     AFG1999  change_EQRM                  
2     AFG2016  change_EQRM                  
3     ALB1996  change_EQRM                  
4     ALB2016  change_EQRM  Implemented EQRM


In [127]:
# now we can drop the variable column because we have that info in the Change column
change_melt = change_melt.drop(columns=['variable'])
print(change_melt.head())

  CountryYear            Change
0     AFG1988                  
1     AFG1999                  
2     AFG2016                  
3     ALB1996                  
4     ALB2016  Implemented EQRM


In [136]:
# creating a series for the next step
x = pd.Series(["Repealed CRDD", "Implemented CRDD", "Repealed EQRM", "Implemented EQRM", "Repealed NODC", "Implemented NODC", "Repealed PRGW", "Implemented PRGW", "Repealed SXHR", "Implemented SXHR", "Repealed NMCN", "Implemented NMCN", "Repealed REGT", "Implemented REGT", "Repealed JOBS", "Implemented JOBS"])

In [140]:
# and remove the rows with empty values in the Change column
final = change_melt.loc[change_melt['Change'].isin(x)]
print(final.head())
print(len(final))

   CountryYear            Change
4      ALB2016  Implemented EQRM
5      DZA1990  Implemented EQRM
9      AGO2000  Implemented EQRM
13     ARG1976  Implemented EQRM
23     AUS1997  Implemented EQRM
663


In [152]:
# now let's bring back the country name and year info
final2 = final.merge(wb3, on='CountryYear')

#quick check
print(final2.head())
print(len(final2))

  CountryYear            Change CountryName CountryCode  Year
0     ALB2016  Implemented EQRM     Albania         ALB  2016
1     DZA1990  Implemented EQRM     Algeria         DZA  1990
2     AGO2000  Implemented EQRM      Angola         AGO  2000
3     AGO2000  Implemented NODC      Angola         AGO  2000
4     ARG1976  Implemented EQRM   Argentina         ARG  1976
663


In [173]:
# and summarize the data for visualization
years = pd.DataFrame(final2['Year'].value_counts())
years.columns=['Count of Changes']
years.index.names = ['Year']
years


,Count of Changes
Year,
2005,25
2004,24
2000,23
1997,23
2008,23
1973,22
2002,22
1988,21
1995,21


In [183]:
countries = pd.DataFrame(final2['CountryName'].value_counts())
countries.columns=['Count of Changes']
countries.index.names = ['Country']
countries


,Count of Changes
Country,
Togo,8
South Africa,8
Lesotho,7
Malta,7
Luxembourg,7
...,...
Papua New Guinea,1
Guinea-Bissau,1
Jordan,1
